<a href="https://colab.research.google.com/github/Diegow7/Create-Repo/blob/main/BooleanWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Information of the Retrieval**

**Homework 2**

**Topic: Basic Boolean Search in Documents**

**Objective**
Expand the simple term search functionality to include Boolean search capabilities. This will allow users to perform more complex queries by combining multiple search terms using Boolean operators.

1. Term Search in Documents without basic boolean

In [52]:
import os
import re
import time
import pandas as pd
from ipywidgets import widgets, VBox, Label

In [24]:
# Ruta de la carpeta
folder_path = "/content/data"

# Lista para almacenar los contenidos de los archivos
file_contents = []

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    # Comprobar si es un archivo de texto
    if filename.endswith(".txt"):
        # Leer el contenido del archivo y añadirlo a la lista
        with open(os.path.join(folder_path, filename), "r") as file:
            file_contents.append(file.read())

In [48]:
import os
import time

# Función para buscar una palabra en los archivos de una carpeta
def search_word_in_files(word, folder):
    start_time = time.time()  # Registro del tiempo de inicio
    books_with_word = {}
    for file in os.listdir(folder):
        if file.endswith('.txt'):
            file_path = os.path.join(folder, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                words = content.lower().split()  # Dividir el contenido en palabras
                occurrences = sum(1 for w in words if word.lower() in w)  # Contar las ocurrencias de la palabra completa
                if occurrences > 0:
                    books_with_word[file] = occurrences
    end_time = time.time()  # Registro del tiempo de finalización
    search_time = end_time - start_time  # Calcula el tiempo transcurrido
    return books_with_word, search_time

# Función para buscar una palabra y mostrar los resultados
def search_word(word_to_search):
    results, search_time = search_word_in_files(word_to_search, "data")
    if results:
        result_str = f"The word '{word_to_search}' is found in the following books:\n"
        for book, occurrences in results.items():
            result_str += f"- {book}: {occurrences} times\n"
        result_str += f"\nSearch time: {search_time:.4f} seconds"  # Añade el tiempo de búsqueda al resultado
        output_label.value = result_str
    else:
        output_label.value = f"The word '{word_to_search}' was not found in any of the books."

# Widget de entrada para la palabra a buscar
search_widget = widgets.Text(
    value='',
    placeholder='Enter word...',
    description='Word to search:',
    disabled=False
)

# Widget de salida para mostrar los resultados
output_label = widgets.Label(
    value='',
    description='Search Results:'
)

# Función para manejar la búsqueda cuando se presiona Enter
def handle_enter(sender):
    search_word(search_widget.value)

search_widget.on_submit(handle_enter)

VBox([search_widget, output_label])

2. Term Search in Documents with basic boolean

In [53]:
def create_boolean_matrix(folder):
    books = []
    words_in_books = {}
    unique_words = set()

    # Expresión regular para eliminar caracteres no deseados
    pattern = re.compile(r"[^\w']")

    # Recorre los archivos de la carpeta
    for file in os.listdir(folder):
        if file.endswith('.txt'):
            books.append(file)
            file_path = os.path.join(folder, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

                # Eliminar caracteres no deseados y dividir en palabras
                words = set(pattern.sub(' ', content).lower().split())

                for word in words:
                    # Ignorar palabras vacías y números
                    if word.isalpha():
                        unique_words.add(word)
                        if word in words_in_books:
                            words_in_books[word].append(1)
                        else:
                            words_in_books[word] = [1 if book == file else 0 for book in books]

    # Construye el DataFrame
    df = pd.DataFrame(words_in_books, index=books).T
    df.index.name = "Words"

    return df

# Crear la matriz booleana
matrix = create_boolean_matrix("data")

# Mostrar la matriz
print(matrix)

# Función para buscar una palabra en la matriz booleana
def search_word_in_matrix(word, matrix):
    start_time = time.time()  # Registro del tiempo de inicio
    if word.lower() in matrix.index:
        books = matrix.loc[word.lower()].loc[matrix.loc[word.lower()] == 1].index.tolist()
    else:
        books = []
    end_time = time.time()  # Registro del tiempo de finalización
    search_time = end_time - start_time  # Calcula el tiempo transcurrido
    return books, search_time

# Función para buscar una palabra y mostrar los resultados
def search_word(word_to_search, matrix):
    books, search_time = search_word_in_matrix(word_to_search, matrix)
    if books:
        result_str = f"The word '{word_to_search}' is found in the following books:\n"
        for book in books:
            result_str += f"- {book}\n"
        result_str += f"\nSearch time: {search_time:.4f} seconds"  # Añade el tiempo de búsqueda al resultado
        output_label.value = result_str
    else:
        output_label.value = f"The word '{word_to_search}' was not found in any of the books."

# Widget de entrada para la palabra a buscar
search_widget = widgets.Text(
    value='',
    placeholder='Enter word...',
    description='Word to search:',
    disabled=False
)

# Widget de salida para mostrar los resultados
output_label = widgets.Label(
    value='',
    description='Search Results:'
)

# Función para manejar la búsqueda cuando se presiona Enter
def handle_enter(sender):
    search_word(search_widget.value, matrix)

search_widget.on_submit(handle_enter)

VBox([search_widget, output_label])

            pg16.txt  pg11.txt
Words                         
fame               1         1
blown              1         1
bedspreads         1         1
shrieks            1         1
silence            1         1
...              ...       ...
unpleasant         0         1
gained             0         1
neighbour          0         1
denial             0         1
belongs            0         1

[6054 rows x 2 columns]


Conclusion

The time is less when we use an array that has the indexes of words and books with its Boolean result, than the iteration that we saw in the first class with the use of a for and if loop.